In [34]:
from llama_index.core import Settings
from llama_index.core.response.notebook_utils import display_response
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.service_context import ServiceContext
from llama_index.core.prompts import PromptTemplate
import os


In [35]:
# llm = Ollama(model="llama3.2-vision:11b", request_timeout=360.0)
# embeding_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

Settings

In [36]:
storage= 'storage'

In [37]:
from llama_index.core import Settings

embedding = HuggingFaceEmbedding(
    model_name="nomic-ai/nomic-embed-text-v1.5", 
    trust_remote_code=True,
    query_instruction="search_query: ",  # Prefijo para consultas
    text_instruction="search_document: "  # Prefijo para documentos
)
llm=Ollama(model="llama3.2:3b", request_timeout=360.0,temperature=0.1)

Settings.embed_model = embedding
Settings.llm = llm



<All keys matched successfully>


Database

In [38]:
from llama_index.core import  VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

documents = SimpleDirectoryReader('documentacion').load_data()

db= chromadb.PersistentClient(path=storage)

chroma_collection = db.get_or_create_collection('ChatCSV')

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Creación o carga del índice
if not os.path.exists(storage):
    # Crear un nuevo índice si no existe almacenamiento previo
    index = VectorStoreIndex.from_documents(
        documents, 
        embedding=embedding, 
        llm=llm, 
        show_progress=True, 
        storage_context=storage_context
    )
    # Guardar índice
    index.storage_context.persist(storage)
else:
    # Cargar índice existente
    index = VectorStoreIndex.from_vector_store(
        vector_store, 
        embedding=embedding, 
        llm=llm, 
        show_progress=True, 
        storage_context=storage_context
    )

In [39]:
qa_prompt_str = (
    "La informacion proveniente de la documentacion es la siguiente.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Dado la informacion brindada en la documentacion, y no tu conocimiento a priori,"
    "Responde la pregunta del operador: {query_str}\n"
)

refine_prompt_str = (
    "Tenemos la oportunidad de refinar la respuesta original "
    "(solo si es necesario) con mas documentacion debajo.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Dado la nueva documentacion, Refina la respuesta original para mejor "
    "respondiendo la pregunta: {query_str}. "
    "Si el contexto no es util, devuelve la respuesta original.\n"
    "Respuesta original: {existing_answer}"
)

In [40]:
from llama_index.core import ChatPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    (
        "system",
        """El asistente es ChatCSV, Creado por Mera Solutions.
        
        Si se presenta un problema matematico, de logia o otro problema proveniente de pensamiento sistematico, ChatCSV piensa a traves de paso a paso antes de dar una respuesta final
        
        Si ChatCSV es consultado de una persona en especifico, este indicara que no posee informacion de personas individuales
        
        ChatCSV es curioso y en caso de no entender poder responder la pregunta, piden que se la reformulen
        
        ChatCSV entiende que hay cosas que no sabe y avisa de lo mismo,
        
        ChatCSV esta recibiendo contexto mediante RAG
        
        ChatCSV siempre brindara la URL indicado en la documentacion al operador
        
        ChatCSV siempre respondera en español, usando la informacion brindada en la documentacion
        
        ChatCSV se encarga de ayudar a un operador telefonico del Centro de Servicios Visa, mientras que este atiende a un cliente al telefono, por lo que debe dar respuestas lo mas certeras posibles, faciles de leer y concretas para que el operador pueda dar una respuesta al cliente
        
        ChatCSV esta siendo conectado con el humano""",
    ),
    ("user", qa_prompt_str),
]
text_qa_template = ChatPromptTemplate.from_messages(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    (
        "system",
        """El asistente es ChatCSV, Creado por Mera Solutions.
        
        Si se presenta un problema matematico, de logia o otro problema proveniente de pensamiento sistematico, ChatCSV piensa a traves de paso a paso antes de dar una respuesta final
        
        Si ChatCSV es consultado de una persona en especifico, este indicara que no posee informacion de personas individuales
        
        ChatCSV es curioso y en caso de no entender poder responder la pregunta, piden que se la reformulen
        
        ChatCSV entiende que hay cosas que no sabe y avisa de lo mismo,
        
        ChatCSV esta recibiendo contexto mediante RAG        
        
        ChatCSV siempre brindara la URL indicado en la documentacion al operador
        
        ChatCSV siempre respondera en español, usando la informacion brindada en la documentacion
        
        ChatCSV se encarga de ayudar a un operador telefonico del Centro de Servicios Visa, mientras que este atiende a un cliente al telefono, por lo que debe dar respuestas lo mas certeras posibles, faciles de leer y concretas para que el operador pueda dar una respuesta al cliente
        
        ChatCSV esta siendo conectado con el humano""",
    ),
    ("user", refine_prompt_str),
]
refine_template = ChatPromptTemplate.from_messages(chat_refine_msgs)


In [41]:

query_engine = index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        llm=llm,
        response_mode='refine'
    )



In [42]:
response = query_engine.query('¿Cómo procedo si el cliente me dice que no le llegó la tarjeta de reposición? Me puedes dar su pagina en gps')
display_response(response)


**`Final Response:`** **Procedimiento para solicitar una tarjeta de reposición y cómo proceder si el cliente dice que no llegó la tarjeta**

Si un cliente te informa que no ha recibido su tarjeta de reposición, es importante recordarle que la tarjeta se generó automáticamente según las condiciones establecidas por el banco emisor. En este caso, hay algunas posibles razones para que la tarjeta no haya sido enviada:

1. **Bloqueo temporal**: Es posible que el sistema haya bloqueado temporalmente la tarjeta debido a una denuncia o incidente relacionado con la tarjeta.
2. **Problemas de envío**: El sistema puede haber tenido problemas para enviar la tarjeta, lo que podría estar causado por un error en el sistema o problemas técnicos.

**Paso 1: Verificar si hay mensajes presionar F22**

Pregúntale al cliente si ha visto alguna pantalla con la leyenda "HAY MENSAJES PRESIONE F22". Esto podría indicar que hay algún mensaje importante que necesita ser atendido.

**Paso 2: Comunica con el banco emisor**

Es fundamental que el cliente se comunique con su banco emisor para obtener más información sobre la situación de su tarjeta y proceder con la reposición, si es necesario. El cliente puede contactar al banco emisor a través del número de teléfono o correo electrónico proporcionado en la documentación.

**Paso 3: Registrar en el campo "Observaciones"**

Si el sistema permite, asegúrate de registrar en el campo "Observaciones" que la tarjeta se generó automáticamente según las condiciones establecidas por el banco emisor.

**Página web del GPS**

La página web del GPS es:

https://gps.prismamediosdepago.com/Lists/Novedades/DispForm.aspx?ID=22727

**Documentación**

La documentación completa se encuentra en el archivo de la siguiente ubicación:

file_path: c:\Users\io6723\Desktop\ChatCSV\documentacion\Novedades - TARJETA DE REPOSICION - CARACTERISTICAS -....txt

Espero que esta información te sea útil. Si tienes alguna otra pregunta o necesitas más ayuda, no dudes en preguntar.

**Nota:** Si el contexto no es util para responder la pregunta del cliente, se devolverá la respuesta original.

Contexto utilizado

In [43]:
for _ in response.source_nodes:
    print('X'*50)
    print(_.text)

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
URL: https://gps.prismamediosdepago.com/Lists/Novedades/DispForm.aspx?ID=22727

Existen otros motivos (ajenos al estado de la tarjeta) que provocan que no se genere una tarjeta de reposición: estas situaciones están definidas por el banco emisor.Ante la denuncia de una tarjeta que no deba ser repuesta por definición bancaria, luego de completar los datos del bloqueo, el sistema no mostrará la pantalla de "Alta de reposición" sino que permanecerá en la 1° pantalla y mostrarála leyenda:"HAY MENSAJES PRESIONE F22"El socio debería contactarse con su banco para gestionar la reposición.Speech:Sr. la reposición queda pendiente, comuníquese con su banco para poder gestionarla.Importante:en el campo "Observaciones" registrar SIN REPO POR DEFINICIÓN DEL BANCO.​
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
URL: https://gps.prismamediosdepago.com/Lists/Novedades/DispForm.aspx?ID=22716

GENERAR LA TARJETA DE REPOSICIÓNAl momento de bloquear u